In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')
from datetime import datetime

In [2]:
pd.options.display.float_format = '{:.2f}'.format

In [3]:
"""
directory = r'D:\RECONCILIATIONS\PAIEMENT\OM_SENEGAL\Données OP\2025\AOUT'

# Trouver le fichier CSV le plus récent dans le répertoire
def get_latest_file(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    latest_file = max(files, key=os.path.getctime)
    return latest_file

# Charger le fichier CSV le plus récent
latest_file_path = get_latest_file(directory)
op = pd.read_csv(latest_file_path,sep='|')
op.sample()
"""


directory = r'D:\RECONCILIATIONS\PAIEMENT\OM_SENEGAL\Données_PMT\2025\NOV'

# Trouver le fichier CSV le plus récent dans le répertoire
def get_latest_file(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    latest_file = max(files, key=os.path.getctime)
    return latest_file

# Charger le fichier CSV le plus récent
latest_file_path = get_latest_file(directory)
pmt = pd.read_csv(latest_file_path)


def pmtt(folder_path):
    # Lister tous les fichiers CSV dans le dossier
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Lire chaque fichier CSV avec le bon séparateur
    df_list = [pd.read_csv(file, sep='|') for file in all_files]

    # Concaténer tous les DataFrames en un seul
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

folder_path = r'D:\RECONCILIATIONS\PAIEMENT\OM_SENEGAL\Données OP\2025\NOV\SOURCE'
op = pmtt(folder_path)

In [4]:
op.head(3)

,Date,Expediteur,Bénéficiaire,Montant,Services,Référence OM,Référence PARTENAIRE,Statut,App Name,CANAL,MESSAGE ERREUR,METADATA
0,10/11/2025 23:42:41,786681749,563777,300,Paiement marchand OTP,MP251110.2342.B34613,ee1f602060354705885716d6badc2194,SUCCESS,bet_sen_sarl_pmt_lonase,API,NaN,[]
1,10/11/2025 23:42:26,771692187,563777,290,Paiement marchand OTP,MP251110.2342.D34982,3f98bd7e76c14b47961a6861d144a8d6,SUCCESS,bet_sen_sarl_pmt_lonase,API,NaN,[]
2,10/11/2025 23:38:10,783851369,563777,5500,Paiement marchand OTP,MP251110.2338.B34304,bbef85dfdcc841a0856163a06aad4155,SUCCESS,bet_sen_sarl_pmt_lonase,API,NaN,[]


In [5]:
op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15242 entries, 0 to 15241
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  15242 non-null  object
 1   Expediteur            15242 non-null  int64 
 2   Bénéficiaire          15242 non-null  int64 
 3   Montant               15242 non-null  int64 
 4   Services              15242 non-null  object
 5   Référence OM          14139 non-null  object
 6   Référence PARTENAIRE  15242 non-null  object
 7   Statut                15242 non-null  object
 8   App Name              15242 non-null  object
 9   CANAL                 15242 non-null  object
 10  MESSAGE ERREUR        1517 non-null   object
 11  METADATA              15242 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.4+ MB


In [6]:

dfpmt = pmt.rename(columns={
        'created_at': 'Created Date',
        'payment_date': 'Payment Date',
        'operator': 'Operator',
        'merchant_name': 'Merchant Name',
        'transaction_id': 'Transaction ID',
        'id_operator': 'ID Opérateur',
        'phone_number': 'Phone Number',
        'amount': 'Montant',
        'fee_amount': 'Fee amount',
        'merchant_amount': 'Merchant amount',
        'statut': 'Statut'
    })

"""
dfpmt=pmt[['Created Date',
         'Payment Date',
         'Operator',
         'Nom du Marchand',
         'Transaction ID',
         'ID Opérateur',
         'Numéro de téléphone',
         'Montant',
         'Fee amount',
         'Merchant amount',
         'Statut'
]]
"""

"\ndfpmt=pmt[['Created Date',\n         'Payment Date',\n         'Operator',\n         'Nom du Marchand',\n         'Transaction ID',\n         'ID Opérateur',\n         'Numéro de téléphone',\n         'Montant',\n         'Fee amount',\n         'Merchant amount',\n         'Statut'\n]]\n"

In [7]:
#CONVERTIR LA DATE EN DATETIME

#EXTRAITRE LA DATE DANS UNE NOUVELLE COLLONNE
dfpmt['DateCourte'] = dfpmt['Created Date'].apply(lambda x: x.split(' ')[0])

In [8]:
def extracted(x):
    parts=x.split(' ')
    return parts[0]
op['DateCourte']=op['Date'].apply(extracted)

In [9]:
op.sample()

,Date,Expediteur,Bénéficiaire,Montant,Services,Référence OM,Référence PARTENAIRE,Statut,App Name,CANAL,MESSAGE ERREUR,METADATA,DateCourte
3682,10/11/2025 21:19:58,774205569,563785,1000,Paiement marchand OTP,MP251110.2119.A19591,0085edd2a4e04d7b938e8f68c333f62b,SUCCESS,service_online_international_pmt_lonase,API,NaN,[],10/11/2025


In [10]:
payin=op[(op['Services']=='Paiement marchand OTP') & (op['Statut']=='SUCCESS')]

In [11]:
duplicated_ids = payin['Référence PARTENAIRE'][payin['Référence PARTENAIRE'].duplicated()]

if not duplicated_ids.empty:
    print(f"⚠️ Il y a {duplicated_ids.nunique()} valeurs dupliquées dans 'ID Marchand'.")
    print("Liste des ID en double :")
    print(duplicated_ids.unique())
else:
    print("✅ Aucun doublon détecté dans 'ID Marchand'.")

✅ Aucun doublon détecté dans 'ID Marchand'.


In [12]:
payin = payin.drop_duplicates(subset='Référence PARTENAIRE')

correspondance_statut_op= payin.set_index('Référence PARTENAIRE')['Statut']
correspondance_date_op = payin.set_index('Référence PARTENAIRE')['DateCourte']
correspondance_idoperator = payin.set_index('Référence PARTENAIRE')['Référence OM']


dfpmt['DATEOP'] = dfpmt['Transaction ID'].map(correspondance_date_op)
dfpmt['STATUTOP'] = dfpmt['Transaction ID'].map(correspondance_statut_op)
dfpmt['IDOPERATOR'] = dfpmt['Transaction ID'].map(correspondance_idoperator)

In [13]:
#NBSI PMT &OMSN
dfpmt['OMSN'] = dfpmt['Transaction ID'].isin(payin['Référence PARTENAIRE']).astype(int)
payin['PMT'] = payin['Référence PARTENAIRE'].isin(dfpmt['Transaction ID']).astype(int)

In [14]:
dfpmt['Nombre']= dfpmt['Montant']
payin['Nombre']= payin['Montant']

In [15]:
df_filteredpmt = dfpmt[dfpmt['OMSN'] == 1]

# Création du tableau croisé dynamique
tcdpmt = pd.pivot_table(
    df_filteredpmt,
    values=['Montant', 'Nombre'],
    index=['DATEOP','Statut'],
    aggfunc={'Nombre': 'count','Montant': 'sum'},
    fill_value=0,
    margins=True,
    margins_name='Total'
)

df_filtered = payin[(payin['PMT'] == 1) | (payin['PMT'] == 0)]

# Création du tableau croisé dynamique
tcdomsn = pd.pivot_table(
    df_filtered,
    values=['Nombre', 'Montant'],
    index=['DateCourte','Statut'],
    aggfunc={'Nombre': 'count','Montant': 'sum' },
    fill_value=0,
    margins=True,
    margins_name='Total'
)

In [16]:
tcdomsn

,,Montant,Nombre
DateCourte,Statut,,
10/11/2025,SUCCESS,52567498,10858
Total,,52567498,10858


In [17]:
tcdpmt

Montant  Nombre
DATEOP     Statut                     
10/11/2025 FAILED      1050.00       1
           SUCCESS 52566448.00   10857
Total              52567498.00   10858

In [18]:
maj_pending = dfpmt.loc[(dfpmt['Statut'] == 'PENDING') & (dfpmt['OMSN'] == 1)]

In [19]:
maj_failed = dfpmt.loc[(dfpmt['Statut'] == 'FAILED') & (dfpmt['OMSN'] == 1)]

In [20]:
trx_succes_a_rechercher = payin.loc[(payin['PMT'] == 0)]

In [21]:
trx_succes_a_rechercher

,Date,Expediteur,Bénéficiaire,Montant,Services,Référence OM,Référence PARTENAIRE,Statut,App Name,CANAL,MESSAGE ERREUR,METADATA,DateCourte,PMT,Nombre


In [22]:
trx_succes_abs_omsn = dfpmt.loc[(dfpmt['OMSN'] == 0) & (dfpmt['Statut'] == 'SUCCESS')]

In [23]:
trx_pending_abs_omsn = dfpmt.loc[(dfpmt['OMSN'] == 0) & (dfpmt['Statut'] == 'PENDING')]

In [24]:

report_date = "20251110"
# Générer automatiquement le chemin de sortie avec la date actuelle
current_date = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = r'D:\RECO\OMSN\PAYIN\RECAPANALYSE\NOV 2025'
output_file_name = f"{current_date}_RAPPORT_ANALYSE_{report_date}.xlsx"
output_file_path = os.path.join(output_dir, output_file_name)

# Création de DataFrames d'exemple
data_pmt = dfpmt
data_op = op
maj_jour_pending_succes = maj_pending
maj_jour_failed_succes = maj_failed
trxenattente_abs = trx_pending_abs_omsn
trx_succes_pmt_abs_omsn=trx_succes_abs_omsn
trx_succes_omsn_abs_pmt = trx_succes_a_rechercher

# Dictionnaire de DataFrames
feuilles = {
    'DATA PMT': data_pmt,
    'data op': data_op,
    'trx_maj_pending_a_succes': maj_jour_pending_succes,
    'trx_maj_failed_a_succes': maj_jour_failed_succes,
    'trxenattente_abs': trxenattente_abs,
    'trx_succes_omsn_abs' :trx_succes_pmt_abs_omsn,
    'trx_succes_omsn_abs_pmt' :trx_succes_a_rechercher
    
}

# Création de l'objet ExcelWriter
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    for nom_feuille, df in feuilles.items():
        df.to_excel(writer, sheet_name=nom_feuille, index=False)

print(f"Fichier Excel avec plusieurs feuilles sauvegardé avec succès: {output_file_path}")

Fichier Excel avec plusieurs feuilles sauvegardé avec succès: D:\RECO\OMSN\PAYIN\RECAPANALYSE\NOV 2025\20251111_100540_RAPPORT_ANALYSE_20251110.xlsx


In [25]:
marchnd=df_filteredpmt.groupby('Merchant Name').agg(
    Nombre=('Montant', 'count'),
    Volume=('Montant', 'sum'))

In [26]:
marchnd

,Nombre,Volume
Merchant Name,,
BETSEN SARL LINEBET,82,175270.00
Lonase 1Xbet,9922,47714570.00
Lonase Melbet,854,4677658.00


In [27]:
mvt_sur_compte=op[(op['Services'] != 'Paiement marchand OTP') & (op['Services'] != "Dépôt d'argent")]
mvt_sur_compte

,Date,Expediteur,Bénéficiaire,Montant,Services,Référence OM,Référence PARTENAIRE,Statut,App Name,CANAL,MESSAGE ERREUR,METADATA,DateCourte


In [28]:
op.Services.unique()

array(['Paiement marchand OTP', "Dépôt d'argent"], dtype=object)